In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(6)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [5]:
Meta.splits_stg1

((#674257) [0,1,2,3,4,5,6,7,8,9...],
 (#78545) [132,133,134,135,136,137,138,139,140,141...])

## Training

In [6]:
name = 'baseline_full_stg1'

In [7]:
dbch = get_data(512, 128, splits=Meta.splits_stg1)
learn = get_learner(dbch, xresnet34)

## Training

In [8]:
do_fit(learn, 20, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.124876,0.153154,0.951881,0.908244,08:04
1,0.111412,0.160502,0.959206,0.908065,08:00
2,0.103698,0.138768,0.958090,0.905519,08:01
3,0.099448,0.121455,0.961430,0.924960,08:02
4,0.097145,0.116398,0.965470,0.928118,08:04
5,0.096724,0.113385,0.967424,0.932090,08:03
6,0.094791,0.119396,0.966866,0.930231,08:02
7,0.090430,0.108540,0.969043,0.935311,08:02
8,0.088001,0.099460,0.970518,0.938583,07:59
9,0.087146,0.101391,0.969795,0.939894,08:03


In [9]:
learn.dbunch = get_data(512, 224, splits=Meta.splits_stg1)
do_fit(learn, 12, 5e-3, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.071752,0.084904,0.974630,0.950742,16:26
1,0.070866,0.085500,0.975371,0.950996,16:15
2,0.070399,0.080020,0.975481,0.952015,16:22
3,0.068074,0.086684,0.974558,0.950105,16:17
4,0.067247,0.084319,0.975184,0.950843,16:16
5,0.065007,0.084734,0.974771,0.950933,16:18
6,0.062094,0.080148,0.976120,0.953504,16:19
7,0.058125,0.080278,0.975863,0.952371,16:18
8,0.054604,0.086552,0.975908,0.952460,16:18
9,0.051881,0.085623,0.976090,0.953161,16:17


In [10]:
learn.dbunch = get_data(256, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 1e-3, freeze=False)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.059987,0.078901,0.976504,0.954357,48:00
1,0.055587,0.078213,0.976822,0.955007,48:30
2,0.048821,0.080487,0.977043,0.955274,48:41
3,0.043575,0.084161,0.976707,0.954523,49:01


## Submission

In [11]:
learn.load(f'runs/{name}-3')

In [12]:
# tst_fns = df_tst.index.values

In [13]:
sub_fn = f'subm/{name}-3'

In [14]:
learn.dbunch = get_test_data(Meta.df_tst, bs=512, sz=384)

In [15]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dbunch = tst_dbch

In [16]:
preds,targs = learn.get_preds()

In [17]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [18]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1-3.csv

In [19]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.72MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [25]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14258280,
 'totalBytes': 27277209,
 'date': '2020-01-22T06:54:06.2Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.084161',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1-3.csv',
 'publicScore': '0.61200',
 'privateScore': '0.06511',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14258280/14258280.raw'}

In [21]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14256645,
 'totalBytes': 27277209,
 'date': '2020-01-22T03:30:30.017Z',
 'description': 'Training on Stage 1 (224), Valid on Stage 2. - 0.087925',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1-2.csv',
 'publicScore': '0.78960',
 'privateScore': '0.07530',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14256645/14256645.raw'}

In [18]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14256314,
 'totalBytes': 27277209,
 'date': '2020-01-22T02:52:23.86Z',
 'description': 'Training on Stage 1, Valid on Stage 2. - 0.087925',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1-2.csv',
 'publicScore': '0.57881',
 'privateScore': '0.08608',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14256314/14256314.raw'}

In [25]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13735979,
 'totalBytes': 27277209,
 'date': '2019-12-10T22:55:30.017Z',
 'description': 'rsna_retro - full - 256 size. Compare to 384 0.064',
 'errorDescription': None,
 'fileName': 'baseline.csv',
 'publicScore': '1.00292',
 'privateScore': '0.08507',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13735979/13735979.raw'}

In [15]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13695014,
 'totalBytes': 27277209,
 'date': '2019-12-08T22:49:07.013Z',
 'description': 'testing rsna_retro submission',
 'errorDescription': None,
 'fileName': 'test_sub.csv',
 'publicScore': '0.95475',
 'privateScore': '0.06444',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13695014/13695014.raw'}